### Imports

In [59]:
import pandas as pd

In [60]:
# pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)

### Leemos y consolidamos los datasets

In [61]:
ventas = pd.read_csv('../Datasets/sell-in.txt', sep='\t')
productos = pd.read_csv('../Datasets/tb_productos.txt', sep='\t')
stock = pd.read_csv('../Datasets/tb_stocks.txt', sep='\t')

ventas_grouped = ventas.groupby(['periodo', 'product_id']).agg({
    'plan_precios_cuidados': 'sum',
    'cust_request_qty': 'sum', # sirve de algo para el modelo? En zulip se me menciona que es la cantidad de solicitudes que hace el cliente, por lo que creo que no
    'cust_request_tn': 'sum',
    'tn': 'sum'
}).reset_index()

merged = pd.merge(productos, stock, on='product_id', how='outer')
final_dataset = pd.merge(merged, ventas_grouped, on=['product_id', 'periodo'], how='outer')

final_dataset.to_csv('../Datasets/final_dataset.csv', sep='\t', index=False)


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:1140: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():


### Analisis exploratorio basico

#### Analisis inicial

In [62]:
final_dataset.head(20)

,cat1,cat2,cat3,brand,sku_size,product_id,periodo,stock_final,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,HC,ROPA LAVADO,Liquido,LIMPIEX,900.00,20280,201810.00,22.60,0.00,138.00,16.07,16.07
1,HC,ROPA LAVADO,Liquido,LIMPIEX,900.00,20280,201811.00,31.93,0.00,126.00,15.04,15.04
2,HC,ROPA LAVADO,Liquido,LIMPIEX,900.00,20280,201812.00,18.26,0.00,131.00,13.59,13.59
3,HC,ROPA LAVADO,Liquido,LIMPIEX,900.00,20280,201901.00,25.81,0.00,126.00,17.24,17.24
4,HC,ROPA LAVADO,Liquido,LIMPIEX,900.00,20280,201902.00,10.13,0.00,155.00,16.74,16.74
5,HC,ROPA LAVADO,Liquido,LIMPIEX,900.00,20280,201903.00,2.14,0.00,168.00,21.87,21.87
6,HC,ROPA LAVADO,Liquido,LIMPIEX,900.00,20280,201904.00,15.16,0.00,124.00,14.50,14.50
7,HC,ROPA LAVADO,Liquido,LIMPIEX,900.00,20280,201905.00,9.28,0.00,106.00,13.36,13.36
8,HC,ROPA LAVADO,Liquido,LIMPIEX,900.00,20280,201906.00,16.81,0.00,103.00,13.53,13.53
9,HC,ROPA LAVADO,Liquido,LIMPIEX,900.00,20280,201907.00,23.11,0.00,193.00,25.50,25.42


In [63]:
final_dataset[final_dataset['cust_request_tn'] != final_dataset['tn']]

,cat1,cat2,cat3,brand,sku_size,product_id,periodo,stock_final,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
9,HC,ROPA LAVADO,Liquido,LIMPIEX,900.00,20280,201907.00,23.11,0.00,193.00,25.50,25.42
19,HC,ROPA LAVADO,Liquido,LIMPIEX,450.00,20180,201902.00,45.05,0.00,337.00,30.76,30.57
20,HC,ROPA LAVADO,Liquido,LIMPIEX,450.00,20180,201903.00,2.46,0.00,402.00,47.85,46.89
24,HC,ROPA LAVADO,Liquido,LIMPIEX,450.00,20180,201907.00,22.41,0.00,307.00,41.04,41.75
25,HC,ROPA LAVADO,Liquido,LIMPIEX,450.00,20180,201908.00,24.32,0.00,254.00,30.06,29.58
...,...,...,...,...,...,...,...,...,...,...,...,...
31556,NaN,NaN,NaN,NaN,NaN,20085,201908.00,NaN,0.00,223.00,174.79,170.37
31557,NaN,NaN,NaN,NaN,NaN,20085,201909.00,NaN,0.00,377.00,255.76,250.33
31558,NaN,NaN,NaN,NaN,NaN,20085,201910.00,NaN,0.00,147.00,175.61,171.16
31561,NaN,NaN,NaN,NaN,NaN,20085,201911.00,NaN,0.00,217.00,236.03,236.03


In [64]:
final_dataset.describe()

,sku_size,product_id,periodo,stock_final,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
count,13984.00,31565.00,31403.00,13851.00,31403.00,31403.00,31403.00,31403.00
mean,491.87,20532.70,201810.81,20.11,1.08,202.79,43.90,42.98
std,963.00,347.20,81.71,56.56,11.96,123.27,114.08,110.28
min,1.00,20001.00,201701.00,-27.31,0.00,1.00,0.00,0.00
25%,90.00,20234.00,201710.00,1.18,0.00,109.00,2.33,2.32
50%,250.00,20491.00,201807.00,5.50,0.00,187.00,9.96,9.89
75%,475.00,20808.00,201904.00,18.04,0.00,283.00,30.66,30.42
max,10000.00,21299.00,201912.00,1562.02,289.00,756.00,2423.71,2295.20


In [65]:
final_dataset[(final_dataset['plan_precios_cuidados'] != 0) & (~final_dataset['plan_precios_cuidados'].isna())]

,cat1,cat2,cat3,brand,sku_size,product_id,periodo,stock_final,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
248,HC,ROPA LAVADO,Liquido,LIMPIEX,800.00,20026,201909.00,27.12,143.00,359.00,210.99,209.49
249,HC,ROPA LAVADO,Liquido,LIMPIEX,800.00,20026,201910.00,68.33,131.00,310.00,376.68,374.11
250,HC,ROPA LAVADO,Liquido,LIMPIEX,800.00,20026,201911.00,42.11,147.00,309.00,313.41,310.41
251,HC,ROPA LAVADO,Liquido,LIMPIEX,800.00,20026,201912.00,72.53,121.00,261.00,235.34,235.10
634,HC,ROPA LAVADO,Liquido,LIMPIEX,400.00,20731,201901.00,66.80,36.00,67.00,3.20,3.20
...,...,...,...,...,...,...,...,...,...,...,...,...
30035,NaN,NaN,NaN,NaN,NaN,20235,201808.00,NaN,104.00,198.00,15.70,15.68
30110,NaN,NaN,NaN,NaN,NaN,20316,201808.00,NaN,154.00,367.00,16.07,15.92
30816,NaN,NaN,NaN,NaN,NaN,20160,201809.00,NaN,94.00,199.00,53.91,53.82
30885,NaN,NaN,NaN,NaN,NaN,20235,201809.00,NaN,168.00,413.00,29.06,28.47


#### Analizo si puede existir alguna venta de un producto en el mismo periodo que no sea plan_precios_cuidados y otras que si

In [66]:
ventas[(ventas['product_id'] == 20026) & (ventas['periodo'] == 201909) & (ventas['plan_precios_cuidados'] == 0)]

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn


In [67]:
def has_both_types(group):
    has_cuidados = (group['plan_precios_cuidados'] != 0).any()
    has_no_cuidados = (group['plan_precios_cuidados'] == 0).any()
    # display(group)
    return has_cuidados and has_no_cuidados

filtered_groups = ventas.groupby(['product_id', 'periodo']).filter(has_both_types)
filtered_groups

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn


Como observo que no existe ningun producto, en ningun periodo, que fuera a su vez precio_cuidoda y que no lo fuera, procedo a modificar esa variable en dataset_final para transformarla en boolean en vez de en suma:

In [68]:
final_dataset['plan_precios_cuidados'] = final_dataset['plan_precios_cuidados'].apply(lambda x: 1 if x > 0 else 0)

In [69]:
final_dataset[(final_dataset['plan_precios_cuidados'] != 0) & (~final_dataset['plan_precios_cuidados'].isna())]

,cat1,cat2,cat3,brand,sku_size,product_id,periodo,stock_final,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
248,HC,ROPA LAVADO,Liquido,LIMPIEX,800.00,20026,201909.00,27.12,1,359.00,210.99,209.49
249,HC,ROPA LAVADO,Liquido,LIMPIEX,800.00,20026,201910.00,68.33,1,310.00,376.68,374.11
250,HC,ROPA LAVADO,Liquido,LIMPIEX,800.00,20026,201911.00,42.11,1,309.00,313.41,310.41
251,HC,ROPA LAVADO,Liquido,LIMPIEX,800.00,20026,201912.00,72.53,1,261.00,235.34,235.10
634,HC,ROPA LAVADO,Liquido,LIMPIEX,400.00,20731,201901.00,66.80,1,67.00,3.20,3.20
...,...,...,...,...,...,...,...,...,...,...,...,...
30035,NaN,NaN,NaN,NaN,NaN,20235,201808.00,NaN,1,198.00,15.70,15.68
30110,NaN,NaN,NaN,NaN,NaN,20316,201808.00,NaN,1,367.00,16.07,15.92
30816,NaN,NaN,NaN,NaN,NaN,20160,201809.00,NaN,1,199.00,53.91,53.82
30885,NaN,NaN,NaN,NaN,NaN,20235,201809.00,NaN,1,413.00,29.06,28.47


In [70]:
# Observo que tambien los Na de plan_precios_cuidados se mapearon a 0
final_dataset[final_dataset['plan_precios_cuidados'].isna()]

,cat1,cat2,cat3,brand,sku_size,product_id,periodo,stock_final,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn


#### Defino la variable target

In [71]:
final_dataset.rename(columns={'tn': 'y'}, inplace=True)
final_dataset.to_csv('../Datasets/final_dataset.csv', sep='\t', index=False)
final_dataset

,cat1,cat2,cat3,brand,sku_size,product_id,periodo,stock_final,plan_precios_cuidados,cust_request_qty,cust_request_tn,y
0,HC,ROPA LAVADO,Liquido,LIMPIEX,900.00,20280,201810.00,22.60,0,138.00,16.07,16.07
1,HC,ROPA LAVADO,Liquido,LIMPIEX,900.00,20280,201811.00,31.93,0,126.00,15.04,15.04
2,HC,ROPA LAVADO,Liquido,LIMPIEX,900.00,20280,201812.00,18.26,0,131.00,13.59,13.59
3,HC,ROPA LAVADO,Liquido,LIMPIEX,900.00,20280,201901.00,25.81,0,126.00,17.24,17.24
4,HC,ROPA LAVADO,Liquido,LIMPIEX,900.00,20280,201902.00,10.13,0,155.00,16.74,16.74
...,...,...,...,...,...,...,...,...,...,...,...,...
31560,NaN,NaN,NaN,NaN,NaN,21169,201910.00,NaN,0,1.00,9.00,9.00
31561,NaN,NaN,NaN,NaN,NaN,20085,201911.00,NaN,0,217.00,236.03,236.03
31562,NaN,NaN,NaN,NaN,NaN,20626,201911.00,NaN,0,1.00,0.26,0.26
31563,NaN,NaN,NaN,NaN,NaN,20085,201912.00,NaN,0,261.00,223.85,221.56


#### Analisis nulos

In [72]:
# TODO
final_dataset.isnull().sum()

cat1                     17581
cat2                     17581
cat3                     17581
brand                    17581
sku_size                 17581
product_id                   0
periodo                    162
stock_final              17714
plan_precios_cuidados        0
cust_request_qty           162
cust_request_tn            162
y                          162
dtype: int64